In [ ]:
from nbdev import *
%nbdev_default_export test

Cells will be exported to nbdev.test,
unless a different module is specified after an export flag: `%nbdev_export special.module`


In [ ]:
%nbdev_export
from nbdev.imports import *
from nbdev.sync import *
from nbdev.export import *

from nbconvert.preprocessors import ExecutePreprocessor

# Extract tests

> The functions that grab the cells containing tests (filtering with potential flags) and execute them

Everything that is not an exported cell is considered a test, so you should make sure your notebooks can all run smoothly (and fast) if you want to use this functionality as the CLI. You can mark some cells with special flags (like slow) to make sure they are only executed when you authorize it. Those flags should be configured in your `settings.ini` (separated by a `|` if you have several of them). 

If `tst_flags=slow|fastai2` in `settings.ini`, you can;
- mark slow tests with the `%nbdev_slow_test` flag
- mark tests that depend on fastai2 with the `%nbdev_fastai2_test` flag. 

To apply a flag to one entire notebook, use the `all` option of the test flag (e.g. `%nbdev_slow_test all`), in one of its cells.

## Detect flags

The following functions detect the cells that should be excluded from the tests (unless their special flag is passed).

In [ ]:
%nbdev_export
_re_all_flag = re.compile("""
# Matches any line that is a test flag with the all option and catches the test flag name in a group:
^%nbdev_  # beginning of line (since re.MULTILINE is passed) and start of magic flag
(\S+)     # the tst flag: group with any non-whitespace chars
_test     # end of magic
[ \t]+all # the all option
\s*       # any number of whitespace
$         # end of line (since re.MULTILINE is passed)
""", re.MULTILINE | re.VERBOSE)

In [ ]:
%nbdev_hide
test_eq('Working', _re_all_flag.search('line1\n%nbdev_Working_test all\nline 3').groups()[0])
test_eq(None, _re_all_flag.search('line1\n%nbdev_Working_test ALL\nline 3'))

In [ ]:
%nbdev_export
def check_all_flag(cells):
    "Check for a cell containing a test flag with the all option and then return said flag"
    for cell in cells:
        if check_re(cell, _re_all_flag): return check_re(cell, _re_all_flag).groups()[0]

In [ ]:
nb = read_nb("04_test.ipynb")
test_eq(None, check_all_flag(nb['cells']))
test_eq('Working', check_all_flag([nbformat.v4.new_code_cell('line1\n%nbdev_Working_test all\nline 3')]))
# only the first "all test" flag will be picked up
test_eq('A1', check_all_flag([nbformat.v4.new_code_cell('%nbdev_A1_test all\n\n%nbdev_Working_test all')]))

In [ ]:
%nbdev_export
def _compile_re_tst_flags(tst_flags):
    "Return a regex pattern to find `tst_flags`"
    if tst_flags is None: return re.compile("^~Dont match ANYTHING~$")
    return re.compile(f"""
# Matches any line with a test flad and catches it in a group:
^%nbdev_        # beginning of line (since re.MULTILINE is passed) and start of magic flag
({tst_flags})   # pipe delimited list of test flags
_test           # end of magic
\s*             # any number of whitespace
$               # end of line (since re.MULTILINE is passed)
""", re.MULTILINE | re.VERBOSE)

class _ReTstFlags():
    def __init__(self): self._re = None
    @property
    def re(self):
        if self._re is None: self._re = _compile_re_tst_flags(Config().get('tst_flags'))
        return self._re

_re_flags = _ReTstFlags()

In [ ]:
%nbdev_hide
#Note: we took _compile_re_tst_flags logic out of _ReTstFlags to make it more testable
def test_compile_re_tst_flags():
    pattern = _compile_re_tst_flags(None)
    test_eq([], pattern.findall('%nbdev_hide\n%nbdev_None_test\n'))
    pattern = _compile_re_tst_flags('')
    test_eq([''], pattern.findall('%nbdev_hide\n%nbdev__test\n'))
    pattern = _compile_re_tst_flags('slow|fastai2')
    test_eq([], pattern.findall('%nbdev_hide\n%nbdev_None_test\n'))
    test_eq([], pattern.findall('%nbdev_hide\n%nbdev__test\n'))
    test_eq(['fastai2'], pattern.findall('%nbdev_hide\n%nbdev_fastai2_test\n'))
    test_eq(['slow'], pattern.findall('%nbdev_hide\n%nbdev_slow_test\n'))
    test_eq(['fastai2','slow'], pattern.findall('%nbdev_hide\n%nbdev_fastai2_test\n%nbdev_slow_test\n'))
    test_eq(['slow','fastai2'], pattern.findall('%nbdev_slow_test\n%nbdev_fastai2_test'))
    test_eq(['slow'], pattern.findall('%nbdev_slow_test\n%nbdev_Fastai2_test'))
test_compile_re_tst_flags()

In [ ]:
%nbdev_export
def get_cell_flags(cell):
    "Check for any special test flag in `cell`"
    if cell['cell_type'] != 'code' or len(Config().get('tst_flags'))==0: return []
    return _re_flags.re.findall(cell['source'])

In [ ]:
test_eq(get_cell_flags({'cell_type': 'code', 'source': "%nbdev_hide\n%nbdev_fastai2_test\n"}), ['fastai2'])
test_eq(get_cell_flags({'cell_type': 'code', 'source': "%nbdev_hide\n"}), [])

## Testing a notebook

In [ ]:
%nbdev_export
class NoExportPreprocessor(ExecutePreprocessor):
    "An `ExecutePreprocessor` that executes cells that are not exported and don't have a flag in `flags`"
    def __init__(self, flags, **kwargs):
        self.flags = flags
        super().__init__(**kwargs)

    def preprocess_cell(self, cell, resources, index):
        if 'source' not in cell or cell['cell_type'] != "code": return cell, resources
        for f in get_cell_flags(cell):
            if f not in self.flags:  return cell, resources
        res = super().preprocess_cell(cell, resources, index)
        return res

In [ ]:
%nbdev_export
def test_nb(fn, flags=None):
    "Execute tests in notebook in `fn` with `flags`"
    os.environ["IN_TEST"] = '1'
    if flags is None: flags = []
    try:
        nb = read_nb(fn)
        all_flag = check_all_flag(nb['cells'])
        if all_flag is not None and all_flag not in flags: return
        mod = find_default_export(nb['cells'])
        ep = NoExportPreprocessor(flags, timeout=600, kernel_name='python3')
        pnb = nbformat.from_dict(nb)
        ep.preprocess(pnb)
    finally: os.environ.pop("IN_TEST")

## Export-

In [ ]:
%nbdev_hide
notebook2script()

Converted 00_export.ipynb.
Converted 01_sync.ipynb.
Converted 02_showdoc.ipynb.
Converted 03_export2html.ipynb.
Converted 04_test.ipynb.
Converted 05_merge.ipynb.
Converted 06_cli.ipynb.
Converted 07_clean.ipynb.
Converted 08_flags.ipynb.
Converted 09_migrate2magic.ipynb.
Converted 99_search.ipynb.
Converted index.ipynb.
Converted tutorial.ipynb.
